In [1]:

import os
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img

In [2]:
import shutil

import numpy as np
base_path = '/Users/whybless/Documents/ai/Aniking'

backup_data = os.path.join(base_path, 'Backup data')
backup_train_data = os.path.join(backup_data, 'train')
backup_val_data = os.path.join(backup_data, 'val')
TRAINING_DIR = os.path.join(base_path, 'clean_jjk_data/train')
VALIDATION_DIR = os.path.join(base_path, 'clean_jjk_data/val')

def copy_and_reset_data(source_dir, destination_dir):
    """Copies data from the source directory to the destination directory,
       deleting the destination directory (if it exists) before copying.

    Args:
        source_dir (str): Path to the source directory.
        destination_dir (str): Path to the destination directory.
    """

    try:
        if os.path.exists(destination_dir):
            shutil.rmtree(destination_dir)
        shutil.copytree(source_dir, destination_dir)
    except OSError as e:
        print(f"Error copying data: {e}")


# Copy train data
copy_and_reset_data(backup_train_data, TRAINING_DIR)

# Copy validation data (using backup_val_data)
copy_and_reset_data(backup_val_data, VALIDATION_DIR)





In [3]:
print('For backup')
print(f"Number of training dirs is {len(os.listdir(backup_train_data))}")
print(f"Number of validation dirs is {len(os.listdir(backup_val_data))}")
print('For data')
print(f"Number of training dirs is {len(os.listdir(TRAINING_DIR))}")
print(f"Number of validation dirs is {len(os.listdir(VALIDATION_DIR))}")

For backup
Number of training dirs is 22
Number of validation dirs is 22
For data
Number of training dirs is 22
Number of validation dirs is 22


In [4]:
char_chosen = "Yuji Itadori"
char_backup_train_files = os.path.join(backup_train_data, 'Yuji Itadori')
print(f'Number of Itadori backup training files is {len(os.listdir(char_backup_train_files))}')
char_train_files = os.path.join(TRAINING_DIR, 'Yuji Itadori')
print(f'Number of Itadori training  files is {len(os.listdir(char_train_files))}')

print(os.listdir(TRAINING_DIR))
for dir in os.listdir(TRAINING_DIR):
    if not dir.startswith('.'):
        char_chosen = f"{dir}"
        char_train_files = os.path.join(TRAINING_DIR, char_chosen)
        print(f'Number of {char_chosen} training char files is {len(os.listdir(char_train_files))}')


Number of Itadori backup training files is 760
Number of Itadori training  files is 760
['Kento Nanami', 'Kiyotaka Ijichi', 'Maki Zenin', 'Yuji Itadori', 'Noritosho Kamo', 'Nobara Kugisaki', 'Momo Nishimiya', '.DS_Store', 'Curse users', 'Mai Zenin', 'Panda', 'Toge Inumaki', 'Mahito', 'Satoru Gojo', 'Junpei Yoshino', 'Sorcerer', 'Sukuna', 'Civilians', 'Curse', 'Megumi Fushiguro', 'Aoi Todo', 'Kasumi Miwa']
Number of Kento Nanami training char files is 126
Number of Kiyotaka Ijichi training char files is 56
Number of Maki Zenin training char files is 141
Number of Yuji Itadori training char files is 760
Number of Noritosho Kamo training char files is 60
Number of Nobara Kugisaki training char files is 308
Number of Momo Nishimiya training char files is 60
Number of Curse users training char files is 41
Number of Mai Zenin training char files is 80
Number of Panda training char files is 21
Number of Toge Inumaki training char files is 57
Number of Mahito training char files is 135
Number 

In [38]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch = 42
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=batch
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=batch
)

Found 3459 images belonging to 21 classes.
Found 862 images belonging to 21 classes.


In [39]:
print(f"val")
class_len = len(validation_generator.class_indices.keys())  # Assuming subdirectories correspond to classes
class_names = list(validation_generator.class_indices.keys())  # Assuming subdirectories correspond to classes
print(f"len = {class_len}")
print(f"Class names: {class_names}")
print(f"train")
class_len = len(train_generator.class_indices.keys())  # Assuming subdirectories correspond to classes
class_names = list(train_generator.class_indices.keys())  # Assuming subdirectories correspond to classes
print(f"len = {class_len}")
print(f"Class names: {class_names}")

val
len = 21
Class names: ['Aoi Todo', 'Civilians', 'Curse', 'Curse users', 'Junpei Yoshino', 'Kasumi Miwa', 'Kento Nanami', 'Kiyotaka Ijichi', 'Mahito', 'Mai Zenin', 'Maki Zenin', 'Megumi Fushiguro', 'Momo Nishimiya', 'Nobara Kugisaki', 'Noritosho Kamo', 'Panda', 'Satoru Gojo', 'Sorcerer', 'Sukuna', 'Toge Inumaki', 'Yuji Itadori']
train
len = 21
Class names: ['Aoi Todo', 'Civilians', 'Curse', 'Curse users', 'Junpei Yoshino', 'Kasumi Miwa', 'Kento Nanami', 'Kiyotaka Ijichi', 'Mahito', 'Mai Zenin', 'Maki Zenin', 'Megumi Fushiguro', 'Momo Nishimiya', 'Nobara Kugisaki', 'Noritosho Kamo', 'Panda', 'Satoru Gojo', 'Sorcerer', 'Sukuna', 'Toge Inumaki', 'Yuji Itadori']


In [41]:
model = tf.keras.Sequential()

# Load the pretrained model with its weights
base_model = tf.keras.applications.ResNet50(
    input_shape=(150, 150, 3),
    weights="imagenet",
    include_top=False
)
# Add the pretrained MobileNet
model.add(base_model)

# Features detector
model.add(layers.GlobalAveragePooling2D())
model.add(layers.BatchNormalization())

# Fully connected layers
model.add(layers.Dense(units=1024, activation="relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(units=1024, activation="relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(units=512, activation="relu"))
model.add(layers.Dropout(0.2))

# Final output : probabilities
model.add(layers.Dense(21, activation="softmax", name="final_output"))

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()  # Print the model summary


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 5, 5, 2048)        23587712  
                                                                 
 global_average_pooling2d_4  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_4 (Bat  (None, 2048)              8192      
 chNormalization)                                                
                                                                 
 dense_22 (Dense)            (None, 1024)              2098176   
                                                                 
 dropout_21 (Dropout)        (None, 1024)              0         
                                                                 
 dense_23 (Dense)            (None, 1024)             

In [42]:


#Early stopping
stop_early = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy", 
    patience=15, 
    mode ="max", 
    verbose=2, 
    restore_best_weights=True
)
# Train the model
history = model.fit(train_generator,
                    epochs=20,
                    verbose=2,
        z            validation_data=validation_generator,
                    callbacks=[stop_early]
)

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/whybless/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/whybless/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/whybless/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/whybless/Library/Python/3.11/lib/python/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/opt/homebrew/Cellar/python@3.11/3.11.7_1/

In [ ]:
#Print results
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.show()
print("")

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.show()

In [ ]:

#Save the model
# Save your model in a variable
model = model.save(os.path.join(base_path, 'jjk_classifier.h5'))

# Inspect parameters
total_params = model.count_params()
num_trainable_params = sum([w.shape.num_elements() for w in model.trainable_weights])

print(f"There are {total_params:,} total parameters in this model.")
print(f"There are {num_trainable_params:,} trainable parameters in this model.")

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model("path/to/your/model.h5")

# User input for image path
image_path = input("Enter the path to your image: ")

# Load and preprocess image
img = load_img(image_path, target_size=(150, 150))
x = img_to_array(img)
# ... Apply further preprocessing steps (normalization, etc.)

# Expand dimension and predict
x = np.expand_dims(x, axis=0)
classes = model.predict(x, batch_size=1)

# Print results
print(image_path)
print(classes)